### CS4622 Mini Kaggle Competition - Classification
#### Author: Sabine Hollatz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import preprocessing, metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

import seaborn as sns

/home/saholly/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [6]:
df_train = pd.read_csv('./data/stock_XY_train.csv')
df_train.head()

,Unnamed: 0,Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Yr,Buy
0,4685,MGEN,2.538000e+06,-0.4125,2.472000e+06,6.600000e+04,1002000.0,10251000.0,1.125300e+07,-1.118700e+07,...,-0.6379,0.0447,0.5950,-0.5153,0.0000,1.8876,0.3534,Healthcare,15,0
1,3142,SO,1.846700e+10,0.0808,1.201200e+10,6.455000e+09,0.0,0.0,1.945000e+09,4.510000e+09,...,0.0704,-0.1432,0.0881,0.0352,0.0636,0.0000,0.0000,Utilities,14,1
2,15877,ONDK,3.509500e+08,0.2047,1.991250e+08,1.518250e+08,53392000.0,94702000.0,1.661700e+08,-1.434500e+07,...,0.0000,0.0000,-0.0639,-0.0175,-0.0826,-0.0935,-0.1790,Financial Services,17,0
3,6977,FFNW,3.392500e+07,-0.0320,0.000000e+00,3.392500e+07,0.0,19533000.0,1.987800e+07,1.404700e+07,...,0.0000,0.0000,0.0458,0.0255,-0.0722,0.0000,0.1465,Financial Services,15,1
4,8237,IFF,3.116350e+09,0.0308,1.720787e+09,1.395563e+09,258863000.0,572518000.0,8.426080e+08,5.529550e+08,...,0.0237,0.0349,0.0851,0.0329,0.2413,0.0519,0.1577,Basic Materials,16,1


In [7]:
df_train = df_train.drop(['Unnamed: 0', 'Ticker', 'Yr'], axis=1)
df_train.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Buy
0,2.538000e+06,-0.4125,2.472000e+06,6.600000e+04,1002000.0,10251000.0,1.125300e+07,-1.118700e+07,141000.0,-1.132800e+07,...,NaN,-0.6379,0.0447,0.5950,-0.5153,0.0000,1.8876,0.3534,Healthcare,0
1,1.846700e+10,0.0808,1.201200e+10,6.455000e+09,0.0,0.0,1.945000e+09,4.510000e+09,835000000.0,3.008000e+09,...,0.0361,0.0704,-0.1432,0.0881,0.0352,0.0636,0.0000,0.0000,Utilities,1
2,3.509500e+08,0.2047,1.991250e+08,1.518250e+08,53392000.0,94702000.0,1.661700e+08,-1.434500e+07,0.0,-1.153400e+07,...,0.0000,0.0000,0.0000,-0.0639,-0.0175,-0.0826,-0.0935,-0.1790,Financial Services,0
3,3.392500e+07,-0.0320,0.000000e+00,3.392500e+07,0.0,19533000.0,1.987800e+07,1.404700e+07,0.0,1.404700e+07,...,0.0000,0.0000,0.0000,0.0458,0.0255,-0.0722,0.0000,0.1465,Financial Services,1
4,3.116350e+09,0.0308,1.720787e+09,1.395563e+09,258863000.0,572518000.0,8.426080e+08,5.529550e+08,52989000.0,5.237170e+08,...,0.1802,0.0237,0.0349,0.0851,0.0329,0.2413,0.0519,0.1577,Basic Materials,1


In [9]:
df_val = pd.read_csv('./data/stock_X_test.csv')
df_val.head()

,Unnamed: 0,Unnamed: 0.1,Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,...,3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Yr
0,0,17685,CMCSA,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,0.000000e+00,6.482200e+10,7.549800e+10,...,0.2348,0.2570,0.0000,0.3426,0.0722,0.7309,0.0000,0.1308,Consumer Cyclical,18
1,1,17686,KMI,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,0.000000e+00,6.010000e+08,3.062000e+09,...,-0.2785,0.0345,-0.0920,-0.0024,0.0076,-0.0137,0.0000,-0.1265,Energy,18
2,2,17687,INTC,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,1.354300e+10,6.750000e+09,2.042100e+10,...,0.0772,0.1989,0.0387,0.0382,0.1014,-0.0169,0.0390,-0.0942,Technology,18
3,3,17688,MU,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,2.141000e+09,8.130000e+08,2.897000e+09,...,0.0000,0.4573,0.1511,0.2275,0.6395,-0.5841,0.1738,0.0942,Technology,18
4,4,17689,GE,1.216150e+11,0.0285,9.546100e+10,2.615400e+10,0.000000e+00,1.811100e+10,4.071100e+10,...,-0.2619,-0.2781,-0.2892,-0.1575,-0.4487,-0.2297,0.0000,0.0308,Industrials,18


In [11]:
df_val = df_val.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Ticker', 'Yr'], axis=1)
df_val.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector
0,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,0.000000e+00,6.482200e+10,7.549800e+10,1.900900e+10,3.542000e+09,1.511100e+10,...,0.1865,0.2348,0.2570,0.0000,0.3426,0.0722,0.7309,0.0000,0.1308,Consumer Cyclical
1,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,0.000000e+00,6.010000e+08,3.062000e+09,3.794000e+09,1.917000e+09,2.196000e+09,...,-0.1421,-0.2785,0.0345,-0.0920,-0.0024,0.0076,-0.0137,0.0000,-0.1265,Energy
2,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,1.354300e+10,6.750000e+09,2.042100e+10,2.331600e+10,-1.260000e+08,2.331700e+10,...,0.0592,0.0772,0.1989,0.0387,0.0382,0.1014,-0.0169,0.0390,-0.0942,Technology
3,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,2.141000e+09,8.130000e+08,2.897000e+09,1.499400e+10,3.420000e+08,1.430300e+10,...,0.0000,0.0000,0.4573,0.1511,0.2275,0.6395,-0.5841,0.1738,0.0942,Technology
4,1.216150e+11,0.0285,9.546100e+10,2.615400e+10,0.000000e+00,1.811100e+10,4.071100e+10,-1.455700e+10,5.059000e+09,-2.177200e+10,...,-0.1408,-0.2619,-0.2781,-0.2892,-0.1575,-0.4487,-0.2297,0.0000,0.0308,Industrials


In [5]:
df_train.dtypes

Unnamed: 0                int64
Ticker                   object
Revenue                 float64
Revenue Growth          float64
Cost of Revenue         float64
                         ...   
R&D Expense Growth      float64
SG&A Expenses Growth    float64
Sector                   object
Yr                        int64
Buy                       int64
Length: 226, dtype: object